In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import trieste
import gpflow
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import math
from util import plotting

In [ ]:
from trieste.objectives.multi_objectives import VLMOP2
from trieste.objectives.utils import mk_observer
from trieste.observer import OBJECTIVE
from trieste.data import Dataset
from trieste.models.gpflow.models import GaussianProcessRegression

from trieste.acquisition import BatchMonteCarloExpectedHypervolumeImprovement
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.bayesian_optimizer import BayesianOptimizer

In [ ]:
from trieste.acquisition.multi_objective.pareto import Pareto, get_reference_point

In [ ]:
from mo_lp.gpr_stack import GPRStack
from mo_lp.mo_penalization import MOLocalPenalizationAcquisitionFunction

## 1d case

In [ ]:
search_space = trieste.space.Box([0], [2*math.pi])

def f1(x):
    return tf.cos(2 * x) + tf.sin(x)

def f2(x):
    return 0.2 * (tf.cos(x) - tf.sin(x)) + 0.3

def f(x):
    return tf.concat([f1(x), f2(x)], axis=-1)

In [ ]:
x_plot = np.linspace(start=search_space.lower[0], stop=search_space.upper[0], num=100)

plt.plot(x_plot, f1(x_plot), label="f1");
plt.plot(x_plot, f2(x_plot), label="f2");
plt.legend();
plt.title("Actual functions");
plt.show();

In [ ]:
def build_stacked_independent_objectives_model(data: Dataset):
    gprs = []
    for idx in range(2):
        single_obj_data = Dataset(
            data.query_points, tf.gather(data.observations, [idx], axis=1)
        )
        variance = tf.math.reduce_variance(single_obj_data.observations)
        kernel = gpflow.kernels.Matern52(variance, tf.constant(0.2, tf.float64))
        gpr = gpflow.models.GPR(single_obj_data.astuple(), kernel, noise_variance=1e-5)
        gpflow.utilities.set_trainable(gpr.likelihood, False)
        gprs.append((GaussianProcessRegression(gpr), 1))

    return GPRStack(*gprs)

In [ ]:
observer = mk_observer(f, OBJECTIVE)

In [ ]:
num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

In [ ]:
num_steps = 3
num_query_points=4

acq_function = BatchMonteCarloExpectedHypervolumeImprovement(sample_size=250).using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)

In [ ]:
result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)
dataset = result.try_get_final_datasets()[OBJECTIVE]

In [ ]:
models = result.try_get_final_models()[OBJECTIVE]._models

In [ ]:
all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();



points_in_objective_space = tf.stack([f1(x_plot), f2(x_plot)], axis=-1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=0)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("True Pareto front");
plt.show();

In [ ]:
acq_function = MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

In [ ]:
result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)
dataset = result.try_get_final_datasets()[OBJECTIVE]

In [ ]:
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();



points_in_objective_space = tf.stack([f1(x_plot), f2(x_plot)], axis=-1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=0)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("True Pareto front");
plt.show();

In [ ]:
import os
from true_pf.generate_true_pareto_fronts import read_true_pf

In [ ]:
from true_pf.generate_true_pareto_fronts import SIMPLE_1D_INPUT_FILENAME

ref_point = get_reference_point(dataset.observations)
true_pf = read_true_pf(os.path.join("true_pf", SIMPLE_1D_INPUT_FILENAME))
ideal_hv = Pareto(true_pf).hypervolume_indicator(ref_point)

hv_regret = []
for i in range(num_initial_points+1, len(dataset.observations)):
    observations = dataset.observations[:i, :]
    observed_hv = Pareto(observations).hypervolume_indicator(ref_point)

    hv_regret.append((ideal_hv - observed_hv).numpy())

hv_regret = np.array(hv_regret)
print(hv_regret)

## 2d case

In [ ]:
search_space = trieste.space.Box([0, 0], [2*math.pi, 2*math.pi])

def f1(input_data):
    x, y = input_data[..., -2], input_data[..., -1]
    z = tf.cos(2.0 * x) * tf.cos(y) + tf.sin(x)
    return z[:, None]

def f2(input_data):
    x, y = input_data[:, -2], input_data[:, -1]
    # changes are made so that the function is between 0 and 1
    z = 1.0 - (tf.cos(x) * tf.cos(y) - tf.sin(x) * tf.sin(y) + 1.0) / 2.0
    return z[:, None]

def f(x):
    return tf.concat([f1(x), f2(x)], axis=-1)

observer = mk_observer(f, OBJECTIVE)

In [ ]:
num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

num_steps = 10
num_query_points=4

acq_function = BatchMonteCarloExpectedHypervolumeImprovement(sample_size=250).using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

In [ ]:
dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

In [ ]:
all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();

grid, xx, yy = plotting.create_grid(search_space.lower, search_space.upper, grid_density=20)

points_in_objective_space = f(grid)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=0)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("True Pareto front");
plt.show();

In [ ]:
acq_function = MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

In [ ]:
dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

In [ ]:
all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();

grid, xx, yy = plotting.create_grid(search_space.lower, search_space.upper, grid_density=20)

points_in_objective_space = f(grid)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=0)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("True Pareto front");
plt.show();

## Trieste integ tests

In [ ]:
def test_multi_objective_optimizer_finds_pareto_front_of_the_VLMOP2_function(
    num_steps, acquisition_rule, convergence_threshold):
    tf.random.set_seed(0)
    search_space = Box([-2, -2], [2, 2])

    def build_stacked_independent_objectives_model(data: Dataset) -> ModelStack:
        gprs = []
        for idx in range(2):
            single_obj_data = Dataset(
                data.query_points, tf.gather(data.observations, [idx], axis=1)
            )
            variance = tf.math.reduce_variance(single_obj_data.observations)
            kernel = gpflow.kernels.Matern52(variance, tf.constant([0.2, 0.2], tf.float64))
            gpr = gpflow.models.GPR(single_obj_data.astuple(), kernel, noise_variance=1e-5)
            gpflow.utilities.set_trainable(gpr.likelihood, False)
            gprs.append((GaussianProcessRegression(gpr), 1))

        return ModelStack(*gprs)

    observer = mk_observer(VLMOP2().objective(), OBJECTIVE)

    initial_query_points = search_space.sample(10)
    initial_data = observer(initial_query_points)

    model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

    dataset = (
        BayesianOptimizer(observer, search_space)
        .optimize(num_steps, initial_data, {OBJECTIVE: model}, acquisition_rule)
        .try_get_final_datasets()[OBJECTIVE]
    )

    # A small log hypervolume difference corresponds to a succesful optimization.
    ref_point = get_reference_point(dataset.observations)

    obs_hv = Pareto(dataset.observations).hypervolume_indicator(ref_point)
    ideal_pf = tf.cast(VLMOP2().gen_pareto_optimal_points(100), dtype=tf.float64)
    ideal_hv = Pareto(ideal_pf).hypervolume_indicator(ref_point)

    assert tf.math.log(ideal_hv - obs_hv) < convergence_threshold

In [ ]:
from trieste.acquisition import (
    BatchMonteCarloExpectedHypervolumeImprovement,
    ExpectedHypervolumeImprovement,
)
from trieste.acquisition.multi_objective.pareto import Pareto, get_reference_point
from trieste.acquisition.optimizer import generate_continuous_optimizer
from trieste.acquisition.rule import (
    AcquisitionRule,
    AsynchronousOptimization,
    EfficientGlobalOptimization,
)
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.data import Dataset
from trieste.models.gpflow import GaussianProcessRegression
from trieste.models.interfaces import ModelStack
from trieste.objectives.multi_objectives import VLMOP2
from trieste.objectives.utils import mk_observer
from trieste.observer import OBJECTIVE
from trieste.space import Box
from trieste.types import TensorType

In [ ]:
input_params = [
    (
            15,
            EfficientGlobalOptimization(
                BatchMonteCarloExpectedHypervolumeImprovement(sample_size=500).using(OBJECTIVE),
                num_query_points=2,
                optimizer=generate_continuous_optimizer(num_initial_samples=500),
            ),
            -3.44,
    ),
    (
            10,
            EfficientGlobalOptimization(
                BatchMonteCarloExpectedHypervolumeImprovement(sample_size=250).using(OBJECTIVE),
                num_query_points=4,
                optimizer=generate_continuous_optimizer(num_initial_samples=500),
            ),
            -3.2095,
    ),
    
    
    # same as above, but with the new LP acq function
    (
            15,
            EfficientGlobalOptimization(
                MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE),
                num_query_points=2,
                optimizer=generate_continuous_optimizer(num_initial_samples=500),
            ),
            -3.44,
    ),
    (
            10,
            EfficientGlobalOptimization(
                MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE),
                num_query_points=4,
                optimizer=generate_continuous_optimizer(num_initial_samples=500),
            ),
            -3.2095,
    ),
]

In [ ]:
for ip in input_params:
    print("-------------------------------")
    print("input:", ip)
    test_multi_objective_optimizer_finds_pareto_front_of_the_VLMOP2_function(ip[0], ip[1], ip[2])

## ZDT3

In [ ]:
search_space = trieste.space.Box([0, 0], [1, 1])

def f1(x):
    return tf.reshape(x[:, 0], (-1, 1))

def f2(x):
    x1 = x[:, 0]
    n = tf.cast(tf.shape(x)[-1], tf.float64)
    g = 1.0 + tf.reduce_sum(x[:, 1:], axis=1) * 9.0 / (n - 1.0)
    h = 1 - tf.sqrt(x1 / g) - x1 / g * tf.sin(10.0 * math.pi * x1)
    return h[:, None]

def f(x):
    return tf.concat([f1(x), f2(x)], axis=-1)

observer = mk_observer(f, OBJECTIVE)

In [ ]:
num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
num_query_points = 5 # 10 runs out of memory and kills the kernel
num_steps = 20

model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

acq_function = BatchMonteCarloExpectedHypervolumeImprovement(sample_size=250).using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

In [ ]:
dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points)
plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();

In [ ]:
num_query_points = 5
num_steps = 20

acq_function = MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

In [ ]:
dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points, figsize=(12,12))

from matplotlib.pyplot import cm
colors = cm.rainbow(np.linspace(0, 1, num_steps))

for i in range(0, num_steps):
    start = num_initial_points + i*num_query_points
    end = num_initial_points + (i+1)*num_query_points
    plt.scatter(f1_model_values[start:end, :], f2_model_values[start:end, :], color=colors[i])

plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();

In [ ]:
num_steps = 100

from trieste.acquisition.function.multi_objective import ExpectedHypervolumeImprovement

acq_function = ExpectedHypervolumeImprovement().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)


dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points, figsize=(12,12))

# from matplotlib.pyplot import cm
# colors = cm.rainbow(np.linspace(0, 1, num_steps))

# for i in range(0, num_steps):
#     start = num_initial_points + i*num_query_points
#     end = num_initial_points + (i+1)*num_query_points
#     plt.scatter(f1_model_values[start:end, :], f2_model_values[start:end, :], color=colors[i])

plt.xlabel("f1");
plt.ylabel("f2");
plt.title("Discovered Pareto front");
plt.show();

## Hartmann and Ackley

In [ ]:
from trieste.objectives.single_objectives import hartmann_6, ackley_5

In [ ]:
# Ackley funciton in Trieste is defined over 5d domain, and we want
def ackley_6(x: TensorType) -> TensorType:
    tf.debugging.assert_shapes([(x, (..., 6))])
    
    x_5d = x[..., :-1]

    return ackley_5(x_5d)

In [ ]:
search_space = trieste.space.Box([0]*6, [1]*6)

def f(x):
    return tf.concat([hartmann_6(x), ackley_6(x)], axis=-1)

observer = mk_observer(f, OBJECTIVE)

In [ ]:
num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
num_query_points = 5
num_steps = 20

model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

acq_function = BatchMonteCarloExpectedHypervolumeImprovement(sample_size=250).using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points, figsize=(12,12))

# from matplotlib.pyplot import cm
# colors = cm.rainbow(np.linspace(0, 1, num_steps))

# for i in range(0, num_steps):
#     start = num_initial_points + i*num_query_points
#     end = num_initial_points + (i+1)*num_query_points
#     plt.scatter(f1_model_values[start:end, :], f2_model_values[start:end, :], color=colors[i])

plt.xlabel("Hartmann");
plt.ylabel("Ackley");
plt.title("BatchMC EHVI");
plt.show();

In [ ]:
num_query_points = 5
num_steps = 20

acq_function = MOLocalPenalizationAcquisitionFunction().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function, num_query_points=num_query_points)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)

dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points, figsize=(12,12))

# from matplotlib.pyplot import cm
# colors = cm.rainbow(np.linspace(0, 1, num_steps))

# for i in range(0, num_steps):
#     start = num_initial_points + i*num_query_points
#     end = num_initial_points + (i+1)*num_query_points
#     plt.scatter(f1_model_values[start:end, :], f2_model_values[start:end, :], color=colors[i])

plt.xlabel("Hartmann");
plt.ylabel("Ackley");
plt.title("MO LP");
plt.show();

In [ ]:
num_steps = 100

from trieste.acquisition.function.multi_objective import ExpectedHypervolumeImprovement

acq_function = ExpectedHypervolumeImprovement().using(OBJECTIVE)
acq_rule = EfficientGlobalOptimization(acq_function)
model = build_stacked_independent_objectives_model(initial_data[OBJECTIVE])

result = BayesianOptimizer(observer, search_space).optimize(num_steps, initial_data, {OBJECTIVE: model}, acq_rule)


dataset = result.try_get_final_datasets()[OBJECTIVE]
models = result.try_get_final_models()[OBJECTIVE]._models

all_query_points = dataset.query_points
f1_model_values, _ = models[0].predict(all_query_points)
f2_model_values, _ = models[1].predict(all_query_points)

points_in_objective_space = tf.concat([f1_model_values, f2_model_values], axis=1)
plotting.plot_mobo_points_in_obj_space(points_in_objective_space, num_init=num_initial_points, figsize=(12,12))

# from matplotlib.pyplot import cm
# colors = cm.rainbow(np.linspace(0, 1, num_steps))

# for i in range(0, num_steps):
#     start = num_initial_points + i*num_query_points
#     end = num_initial_points + (i+1)*num_query_points
#     plt.scatter(f1_model_values[start:end, :], f2_model_values[start:end, :], color=colors[i])

plt.xlabel("Hartmann");
plt.ylabel("Ackley");
plt.title("EHVI");
plt.show();